In [1]:
#from fastai.vision.transform import get_transforms
from fastai.metrics import error_rate
from pathlib import Path
from fastai.vision.all import *

In [2]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
data_dir = '../input/car-data-train'
path=Path(data_dir)
path

In [4]:
batch_tfms = [*aug_transforms(), Normalize.from_stats(*imagenet_stats), Saturation(p=1,draw=0)]
item_tfms = Resize((300, 300), method=ResizeMethod.Crop)

data = ImageDataLoaders.from_folder(path, bs=16, valid_pct=0.2, batch_tfms=batch_tfms,
                                  item_tfms=item_tfms, num_workers=4)

In [5]:
data.show_batch()

In [8]:
learn = Learner(data, models.xresnet.xse_resnext101(n_out=data.c, pretrained=True), metrics=[error_rate, accuracy], model_dir="/tmp")

In [ ]:
learn.fine_tune(10)

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9)

In [ ]:
learn.save('/kaggle/wokring/stage-1')

Unfreezing, fine-tuning, and learning rates

In [ ]:
learn.fit_one_cycle(10,3e-3)

In [ ]:
learn.save('stage-2')

In [ ]:
learn.unfreeze()
learn.lr_find()


In [ ]:
learn.fine_tune(10,6e-5)

In [ ]:
learn.save('stage-3')

In [ ]:
learn.summary()

In [ ]:
import requests
img = requests.get('https://pictures.topspeed.com/IMG/crop/201203/2012-geneva-motor-sh-3_800x0w.jpg')
learn.predict(img.content)


In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(30,25))

In [ ]:
learn.export('/kaggle/working/model.pkl')